In [6]:
import os

import base64
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
token = ""
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o"


In [1]:
import hashlib

# e.g. blobpath = https://openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken
blobpath = "openaipublic.blob.core.windows.net/encodings/cl100k_base.tiktoken"
cache_key = hashlib.sha1(blobpath.encode()).hexdigest()
print(cache_key)

b611642958a8bc2bbdf59bca895a629634228ed9


In [ ]:

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)
id= "10005"
base_path = os.getcwd()
audio_path = os.path.join(base_path, "data", f"{id}", "Audio", id, ".mp3")
audio_file= open(audio_path, "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file
)

print(transcription.text)

InternalServerError: <html>
<head><title>500 Internal Server Error</title></head>
<body>
<center><h1>500 Internal Server Error</h1></center>
<hr><center>nginx</center>
</body>
</html>

In [11]:

def get_image_data_url(image_file: str, image_format: str) -> str:
    """
    Helper function to converts an image file to a data URL string.

    Args:
        image_file (str): The path to the image file.
        image_format (str): The format of the image file.

    Returns:
        str: The data URL of the image.
    """
    try:
        with open(image_file, "rb") as f:
            image_data = base64.b64encode(f.read()).decode("utf-8")
    except FileNotFoundError:
        print(f"Could not read '{image_file}'.")
        exit()
    return f"data:image/{image_format};base64,{image_data}"


In [12]:
client = OpenAI(
    base_url=endpoint,
    api_key=token,
)


In [13]:
system_prompt = '''You are an image analysis assistant. Provide brief, precise descriptions of meeting screenshots, focusing on both visual elements and content. Keep responses concise and factual.'''

user_prompt = '''Describe this meeting screenshot concisely, including:

1. Layout: screen composition, UI elements, meeting controls
2. Text Content: 
- All visible text from slides/charts
- Participant names
- Chat messages
3. Visual Elements:
- Charts: type, data, key metrics
- Slides: design, graphics
4. Meeting Info:
- Participant count
- Timestamps
- Platform features

Keep descriptions brief but include all visible text and numerical data.'''

In [14]:
id = 10004
base_path = os.getcwd()
screenshots_path = os.path.join(base_path, "data", f"{id}", "Screenshots", "screenshot_0121.png")
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {
                      #miniproject/backend/src/data/10004/Screenshots/screenshot_0121.png
                        "url": get_image_data_url(screenshots_path, "png"),
                        "detail": "low"
                    },
                },
            ],
        },
    ],
    model=model_name,
)

print(response.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 71496 seconds before retrying.', 'details': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 71496 seconds before retrying.'}}

In [16]:
def getDescriptions(id: str):
    descriptions = []
    # base_path = os.path.dirname(__file__)
    base_path = os.getcwd()
    
    screenshots_path = os.path.join(base_path, "data", f"{id}", "Screenshots")
    print(screenshots_path)
    i = 0
    for screenshot in os.listdir(screenshots_path):
        if screenshot.endswith(".png"):
            i += 1
            image_path = os.path.join(screenshots_path, screenshot, ".png")
            print(image_path)
getDescriptions("10004")

/home/himan/projects/miniproject/backend/src/data/10004/Screenshots
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0278.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0209.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0332.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0410.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0217.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0287.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0315.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0246.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0258.png/.png
/home/himan/projects/miniproject/backend/src/data/10004/Screenshots/screenshot_0221.png/.png
/h

In [2]:
# %pip install whisperx
import whisperx
import gc 

device = "cuda" 
audio_file = "/home/himan/projects/miniproject/backend/data/10003/Audio/10003.mp3"
batch_size = 12 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

# save model to local path (optional)
model_dir = "/home/himan/projects/miniproject/backend/app/models"
model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
gc.collect()
import torch
torch.cuda.empty_cache()
del model

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment

# delete model if low on GPU resources
gc.collect()
torch.cuda.empty_cache()
del model_a

# 3. Assign speaker labels
# diarize_model = whisperx.DiarizationPipeline(use_auth_token=YOUR_HF_TOKEN, device=device)

# add min/max number of speakers if known
# diarize_segments = diarize_model(audio)
# diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

# result = whisperx.assign_word_speakers(diarize_segments, result)
# print(diarize_segments)
# print(result["segments"]) # segments are now assigned speaker IDs

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../miniconda3/envs/newenv/lib/python3.11/site-packages/whisperx/assets/pytorch_model.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.
No language specified, language will be first be detected for each audio file (increases inference time).


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../miniconda3/envs/newenv/lib/python3.11/site-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


/home/himan/miniconda3/envs/newenv/lib/python3.11/site-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


: 

In [ ]:
from whispercpp import Whisper

w = Whisper('tiny')

result = w.transcribe("/home/himan/projects/miniproject/backend/data/10003/Audio/10003.mp3")
text = w.extract_text(result)

In [1]:

from faster_whisper import WhisperModel

# File path for the audio file
audio_path = "/home/himan/projects/miniproject/backend/data/10003/Audio/10003.mp3"  # Replace with your file path
subprocess.run(["ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a", audio_path], check=True)
# Initialize the Faster Whisper model
model = WhisperModel("large-v2", device="cuda", compute_type="float16")  # Optimized for RTX GPUs

# Transcribe the audio
segments, info = model.transcribe(
    audio_path,
    beam_size=5,             # Improve accuracy with beam search
    vad_filter=True,
)

# Print audio info
print(f"Audio Info: Duration: {info.duration:.2f} seconds, Language: {info.language}")

# Save transcription to a file
with open("transcription.txt", "w", encoding="utf-8") as f:
    for segment in segments:
        f.write(f"[{segment.start:.2f} - {segment.end:.2f}]: {segment.text}\n")

print("Transcription saved to transcription.txt")

/home/himan/miniconda3/envs/newenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 